<a href="https://colab.research.google.com/github/BStricks/SeaShips/blob/master/bayes_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pomegranate
from pomegranate import *
from shapely.geometry import box

     |████████████████████████████████| 3.3MB 2.7MB/s 
  Created wheel for pomegranate: filename=pomegranate-0.13.0-cp36-cp36m-linux_x86_64.whl size=11007230 sha256=1cf80373e4f0d3c1d232646d3ebb28127564d931f8bd5273084c2d771b227fd1
  Stored in directory: /root/.cache/pip/wheels/f4/b0/21/635932e79ad11c419b90b074ffaf90c04bfaecf134e66c3a06
Successfully built pomegranate


In [0]:

def generateEvent(ais, image=None, text=None):
  #read in jsons and get characteristics
  if ais['PORT_NAME'] == 'SAN DIEGO':

    if -120.3442 <= ais['LAT'] <= -117.6526 and 32.3707 <= ais['LON'] <= 33.2933 :
      position = 'A'
    else:
      position = 'N'
    
    if 0 <= ais['SPEED'] <= 0.8 :
      speed = 'A'
    else:
      speed = 'N'

    if 290 <= ais['HEADING'] <= 350 :
      heading = 'A'
    else:
      heading = 'N'

  elif ais['PORT_NAME'] == 'NORFOLK':

    if -75.7837 <= ais['LAT'] <= -75.0256 and 35.7457 <= ais['LON'] <= 36.7917 :
      position = 'A'
    else:
      position = 'N'
    
    if 0 <= ais['SPEED'] <= 0.8 :
      speed = 'A'
    else:
      speed = 'N'

    if 300 <= ais['HEADING'] <= 340 :
      heading = 'A'
    else:
      heading = 'N'
  
  boat_overlap = 'N'
  if image:
    if len(image['ships']) > 1:
      polygon_list = []
      boat_overlap = 'N'

      for bbox in image['ships']:
        x_min = int(bbox['x_min'])
        y_min = int(bbox['y_min'])
        x_max = int(bbox['x_max'])
        y_max = int(bbox['y_max'])
        rect = box(x_min, y_min, x_max, y_max, ccw=True)
        polygon_list.append(rect)

        for i in range(0, len(polygon_list)):
          for a in range(i+1,len(polygon_list)):
              distance = polygon_list[i].distance(polygon_list[a])
              if distance <= 350.0:
                boat_overlap = 'A'
    else:
      boat_overlap = 'N'

  civilian_sighting = 'N'
  if text:
    if text['VESSEL_NAME'] is not None:
      civilian_sighting = 'A'
    else:
      civilian_sighting = 'N'

  return heading, speed, position, boat_overlap, civilian_sighting



def bayes_network(heading, speed, position, boat_overlap, civilian_sighting):
  heading_dis = DiscreteDistribution({'A': 1./5, 'N': 4./5})
  speed_dis = DiscreteDistribution({'A': 1./5, 'N': 4./5})
  position_dis = DiscreteDistribution({'A': 2./5, 'N': 3./5})
  boat_overlap_dis = DiscreteDistribution({'A': 4./5, 'N': 1./5})
  civilian_sighting_dis = DiscreteDistribution({'A': 4./5, 'N': 1./5})

  event_dis = ConditionalProbabilityTable(
      [['A', 'A', 'A', 'A', 'A','Rendezvous', 1],
       ['A', 'A', 'A', 'A', 'N','Rendezvous', 1],
       ['A', 'A', 'A', 'N', 'A','Rendezvous', 0],
       ['A', 'A', 'A', 'N', 'N','Rendezvous', 0],
       ['A', 'A', 'N', 'A', 'A','Rendezvous', 1],
       ['A', 'A', 'N', 'A', 'N','Rendezvous', 1],
       ['A', 'N', 'A', 'A', 'A','Rendezvous', 1],
       ['A', 'N', 'A', 'A', 'N','Rendezvous', 1],
       ['N', 'A', 'A', 'A', 'A','Rendezvous', 1],
       ['N', 'A', 'A', 'A', 'N','Rendezvous', 1],
       ['N', 'N', 'A', 'A', 'A','Rendezvous', 1],
       ['N', 'N', 'A', 'A', 'N','Rendezvous', 1],
       ['N', 'A', 'N', 'A', 'A','Rendezvous', 1],
       ['N', 'A', 'N', 'A', 'N','Rendezvous', 1],
       ['N', 'A', 'A', 'N', 'A','Rendezvous', 0],
       ['N', 'A', 'A', 'N', 'N','Rendezvous', 0],
       ['A', 'N', 'N', 'A', 'A','Rendezvous', 1],
       ['A', 'N', 'N', 'A', 'N','Rendezvous', 1],
       ['A', 'N', 'A', 'N', 'A','Rendezvous', 0],
       ['A', 'N', 'A', 'N', 'N','Rendezvous', 0],
       ['A', 'A', 'N', 'N', 'A','Rendezvous', 0],
       ['A', 'A', 'N', 'N', 'N','Rendezvous', 0],
       ['A', 'N', 'N', 'N', 'A','Rendezvous', 0],
       ['A', 'N', 'N', 'N', 'N','Rendezvous', 0],
       ['N', 'A', 'N', 'N', 'A','Rendezvous', 0],
       ['N', 'A', 'N', 'N', 'N','Rendezvous', 0],
       ['N', 'N', 'A', 'N', 'A','Rendezvous', 0],
       ['N', 'N', 'A', 'N', 'N','Rendezvous', 0],
       ['N', 'N', 'N', 'A', 'A','Rendezvous', 0.9],
       ['N', 'N', 'N', 'A', 'N','Rendezvous', 1],
       ['N', 'N', 'N', 'N', 'A','Rendezvous', 0],
       ['N', 'N', 'N', 'N', 'N','Rendezvous', 0],
       
       ['A', 'A', 'A', 'A', 'A','Non-civlian Transit', 0],
       ['A', 'A', 'A', 'A', 'N','Non-civlian Transit', 0],
       ['A', 'A', 'A', 'N', 'A','Non-civlian Transit', 0.9],
       ['A', 'A', 'A', 'N', 'N','Non-civlian Transit', 0.8],
       ['A', 'A', 'N', 'A', 'A','Non-civlian Transit', 0],
       ['A', 'A', 'N', 'A', 'N','Non-civlian Transit', 0],
       ['A', 'N', 'A', 'A', 'A','Non-civlian Transit', 0],
       ['A', 'N', 'A', 'A', 'N','Non-civlian Transit', 0],
       ['N', 'A', 'A', 'A', 'A','Non-civlian Transit', 0],
       ['N', 'A', 'A', 'A', 'N','Non-civlian Transit', 0],
       ['N', 'N', 'A', 'A', 'A','Non-civlian Transit', 0],
       ['N', 'N', 'A', 'A', 'N','Non-civlian Transit', 0],
       ['N', 'A', 'N', 'A', 'A','Non-civlian Transit', 0],
       ['N', 'A', 'N', 'A', 'N','Non-civlian Transit', 0],
       ['N', 'A', 'A', 'N', 'A','Non-civlian Transit', 0.9],
       ['N', 'A', 'A', 'N', 'N','Non-civlian Transit', 0.5],
       ['A', 'N', 'N', 'A', 'A','Non-civlian Transit', 0],
       ['A', 'N', 'N', 'A', 'N','Non-civlian Transit', 0],
       ['A', 'N', 'A', 'N', 'A','Non-civlian Transit', 0.9],
       ['A', 'N', 'A', 'N', 'N','Non-civlian Transit', 0.3],
       ['A', 'A', 'N', 'N', 'A','Non-civlian Transit', 0.9],
       ['A', 'A', 'N', 'N', 'N','Non-civlian Transit', 0.3],
       ['A', 'N', 'N', 'N', 'A','Non-civlian Transit', 0.9],
       ['A', 'N', 'N', 'N', 'N','Non-civlian Transit', 0.3],
       ['N', 'A', 'N', 'N', 'A','Non-civlian Transit', 0.9],
       ['N', 'A', 'N', 'N', 'N','Non-civlian Transit', 0.35],
       ['N', 'N', 'A', 'N', 'A','Non-civlian Transit', 0.9],
       ['N', 'N', 'A', 'N', 'N','Non-civlian Transit', 0.2],
       ['N', 'N', 'N', 'A', 'A','Non-civlian Transit', 0.1],
       ['N', 'N', 'N', 'A', 'N','Non-civlian Transit', 0],
       ['N', 'N', 'N', 'N', 'A','Non-civlian Transit', 0.9],
       ['N', 'N', 'N', 'N', 'N','Non-civlian Transit', 0],

       ['A', 'A', 'A', 'A', 'A','Normal Civilian Transit', 0],
       ['A', 'A', 'A', 'A', 'N','Normal Civilian Transit', 0],
       ['A', 'A', 'A', 'N', 'A','Normal Civilian Transit', 0.1],
       ['A', 'A', 'A', 'N', 'N','Normal Civilian Transit', 0.2],
       ['A', 'A', 'N', 'A', 'A','Normal Civilian Transit', 0],
       ['A', 'A', 'N', 'A', 'N','Normal Civilian Transit', 0],
       ['A', 'N', 'A', 'A', 'A','Normal Civilian Transit', 0],
       ['A', 'N', 'A', 'A', 'N','Normal Civilian Transit', 0],
       ['N', 'A', 'A', 'A', 'A','Normal Civilian Transit', 0],
       ['N', 'A', 'A', 'A', 'N','Normal Civilian Transit', 0],
       ['N', 'N', 'A', 'A', 'A','Normal Civilian Transit', 0],
       ['N', 'N', 'A', 'A', 'N','Normal Civilian Transit', 0],
       ['N', 'A', 'N', 'A', 'A','Normal Civilian Transit', 0],
       ['N', 'A', 'N', 'A', 'N','Normal Civilian Transit', 0],
       ['N', 'A', 'A', 'N', 'A','Normal Civilian Transit', 0.1],
       ['N', 'A', 'A', 'N', 'N','Normal Civilian Transit', 0.5],
       ['A', 'N', 'N', 'A', 'A','Normal Civilian Transit', 0],
       ['A', 'N', 'N', 'A', 'N','Normal Civilian Transit', 0],
       ['A', 'N', 'A', 'N', 'A','Normal Civilian Transit', 0.1],
       ['A', 'N', 'A', 'N', 'N','Normal Civilian Transit', 0.7],
       ['A', 'A', 'N', 'N', 'A','Normal Civilian Transit', 0.1],
       ['A', 'A', 'N', 'N', 'N','Normal Civilian Transit', 0.7],
       ['A', 'N', 'N', 'N', 'A','Normal Civilian Transit', 0.1],
       ['A', 'N', 'N', 'N', 'N','Normal Civilian Transit', 0.7],
       ['N', 'A', 'N', 'N', 'A','Normal Civilian Transit', 0.1],
       ['N', 'A', 'N', 'N', 'N','Normal Civilian Transit', 0.65],
       ['N', 'N', 'A', 'N', 'A','Normal Civilian Transit', 0.1],
       ['N', 'N', 'A', 'N', 'N','Normal Civilian Transit', 0.8],
       ['N', 'N', 'N', 'A', 'A','Normal Civilian Transit', 0.1],
       ['N', 'N', 'N', 'A', 'N','Normal Civilian Transit', 0],
       ['N', 'N', 'N', 'N', 'A','Normal Civilian Transit', 0.1],
       ['N', 'N', 'N', 'N', 'N','Normal Civilian Transit', 1]], [heading_dis, speed_dis, position_dis, boat_overlap_dis, civilian_sighting_dis])
  
  s1 = Node(heading_dis, name="heading")
  s2 = Node(speed_dis, name="speed")
  s3 = Node(position_dis, name="position")
  s4 = Node(boat_overlap_dis, name="boat_overlap")
  s5 = Node(civilian_sighting_dis, name="civilian_sighting")
  s6 = Node(event_dis, name="event")
  
  model = BayesianNetwork("ship_event_reasoning")
  model.add_states(s1, s2, s3, s4, s5, s6)
  model.add_edge(s1, s6)
  model.add_edge(s2, s6)
  model.add_edge(s3, s6)
  model.add_edge(s4, s6)
  model.add_edge(s5, s6)
  model.bake()
  pred = model.predict([[heading, speed, position, boat_overlap, civilian_sighting, None]])
  if pred[0][5] == 'Normal Civilian Transit':
    non_pred_1 = 'Non-civlian Transit'
    non_pred_2 = 'Rendezvous'
  elif pred[0][5] == 'Non-civlian Transit':
    non_pred_1 = 'Normal Civilian Transit'
    non_pred_2 = 'Rendezvous'
  elif pred[0][5] == 'Rendezvous':
    non_pred_1 = 'Non-civlian Transit'
    non_pred_2 = 'Normal Civilian Transit'
  prob = model.probability(pred)
  prob1 = model.probability([[heading, speed, position, boat_overlap, civilian_sighting, non_pred_1]])
  prob2 = model.probability([[heading, speed, position, boat_overlap, civilian_sighting, non_pred_2]])
  prob = prob/(prob+prob1+prob2)
  prob = "{:.2f}%".format(prob*100)
  return pred, prob, heading, speed, position, boat_overlap, civilian_sighting
          
def event_reasoning(ais, image=None, text=None):  
  position, speed, heading, boat_overlap, civilian_sighting = generateEvent(ais,image)
  pred, prob, heading, speed, position, boat_overlap, civilian_sighting = bayes_network(position, speed, heading, boat_overlap, civilian_sighting)
  return {'prediction':pred[0][5], 'probability':prob}
          
          
          

In [0]:
my_ais = {
    "HEADING": 216.0,
    "LAT": -76.4352,
    "LON": 36.5364,
    "MMSI": "367442020",
    "MOVE_TYPE": "0",
    "PORT_ID": "136",
    "PORT_NAME": "NORFOLK",
    "SHIPNAME": "JACK HOLLAND",
    "SHIP_ID": "445360",
    "SPEED": 11.96,
    "TIMESTAMP_LT": "2020-04-14T02:29:00",
    "TIMESTAMP_UTC": "2020-04-14T06:29:00",
    "TYPE_NAME": "ORE_CARRIER",
    "UNLOCODE": "USORF"
}
my_dict = event_reasoning(my_ais)

In [15]:
print(my_dict)

{'prediction': 'Normal Civilian Transit', 'probability': '100.00%'}
